# Prepare the jupyterlite environemt
These steps are specific to the tutorial for running a python kernel written in web assembly. When running a regular python kernel, these can be skipped.

## install dependencies 

We dont install mongodb because it is not available in pure python so cannot be used in pyolite.
instead we use a pandas dataframe for demonstration but all features work the same with a mongodb collection as a datasource

In [ ]:
import piplite

await piplite.install('xedocs')

## Fetch some sample data
We must use the js fetch because the python ssl library is not compatible with pyolite.
If we were in native python we could just supply the url to xedocs as the datasource and it would fetch the data automatically for us.

In [ ]:
import io
import pandas as pd
from js import fetch

url = "https://raw.githubusercontent.com/XENONnT/xedocs/master//test_data/pmt_gains_random_data.csv"

r = await fetch(url)
data = await r.text()
f = io.StringIO(data)

df = pd.read_csv(f)
df['time'] = pd.to_datetime(df['time'])

In [ ]:

import datetime
import xedocs

# Create an arbitrary datetime object to demonstrate time based queries 
dtime = datetime.datetime(2022, 4, 4, 8, 1, 1)

# here if we had access to the runs db (utilix installed), we could also pass run_id=XXXXX instead of the time
# and it would use the center time of the run.
xedocs.PmtGain.find_one(df, version='ONLINE', time=dtime, pmt=444)


# RemoteFrame: pandas/xarray interface
For convenience additional query APIs are implemented inspired for the pandas and xarray packages. Most of these methods return a padnas dataframe with the requested data selection.

In [ ]:
# The straxen.frames namespace holds a collection
# of remote frames for all defined corrections

gains_rf = xedocs.PmtGain.rframe(df)

In [ ]:
# List the index names
gains_rf.index_names

### xarray api

In [ ]:
# calling the .sel() method returns a pandas
# dataframe with the selection result

df = gains_rf.sel(detector='tpc', version='ONLINE', time=dtime).df
df

### pandas api

In [ ]:
# pandas style multi-indexing also returns a pandas
# dataframe with the selection result

df = gains_rf.loc['ONLINE',dtime,'tpc'].df
df

#### Scalar lookup

In [ ]:
# pandas api

gains_rf.at[('ONLINE',dtime,'tpc',344), 'value']

In [ ]:
# simple callable

gains_rf('value', detector='tpc', version='ONLINE', time=dtime, pmt=344)